# Modeling Notebook

In [32]:
import pandas as pd, matplotlib.pyplot as plt, numpy as np
from collections import defaultdict
import datetime, random
from collections import Counter
from scipy import sparse
from surprise import Reader, Dataset, accuracy
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split

import numpy as np

First I'm going to make a quick way to get movie titles from movie ids

In [28]:
movies = pd.read_csv('../../../data/movies.csv')
decoder = dict(zip(movies.movieId, movies.title))

just a check

In [46]:
decoder[33312]

'Cocoanuts, The (1929)'

In [45]:
movies.sample(10)

,movieId,title,genres
5892,33312,"Cocoanuts, The (1929)",Comedy|Musical
6337,48982,Flushed Away (2006),Animation|Comedy
2597,3473,Jonah Who Will Be 25 in the Year 2000 (Jonas q...,Comedy
7321,77421,Cyrus (2010),Comedy|Drama|Romance
7456,81591,Black Swan (2010),Drama|Thriller
8692,122906,Black Panther (2017),Action|Adventure|Sci-Fi
6130,43684,Something New (2006),Comedy|Drama|Romance
736,956,Penny Serenade (1941),Drama|Romance
2810,3755,"Perfect Storm, The (2000)",Drama|Thriller
3784,5292,Slap Shot (1977),Comedy


Let's make a reader object to read the ratings csv

In [2]:

reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1, rating_scale=(.5,5))
data = Dataset.load_from_file('../../../data/ratings.csv', reader)

Let's make a test split and fit an SVD model to our training setf

In [7]:
trainset, testset = train_test_split(data, test_size=.25)

algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.8736


0.873600501204693

In [12]:

[pred for pred in predictions if pred[0] == '561']

[Prediction(uid='561', iid='106489', r_ui=3.5, est=3.039203911227965, details={'was_impossible': False}),
 Prediction(uid='561', iid='6365', r_ui=2.5, est=3.021105033397584, details={'was_impossible': False}),
 Prediction(uid='561', iid='204', r_ui=1.5, est=2.566455566572121, details={'was_impossible': False}),
 Prediction(uid='561', iid='1974', r_ui=3.0, est=2.790142598739554, details={'was_impossible': False}),
 Prediction(uid='561', iid='1376', r_ui=4.0, est=3.0623040799954615, details={'was_impossible': False}),
 Prediction(uid='561', iid='6502', r_ui=4.0, est=3.7917599214459834, details={'was_impossible': False}),
 Prediction(uid='561', iid='3499', r_ui=3.0, est=4.057866807892108, details={'was_impossible': False}),
 Prediction(uid='561', iid='82461', r_ui=3.0, est=2.931212436927815, details={'was_impossible': False}),
 Prediction(uid='561', iid='1129', r_ui=4.0, est=3.6281831369963164, details={'was_impossible': False}),
 Prediction(uid='561', iid='41569', r_ui=3.0, est=3.2508878

In [10]:
predictions

[Prediction(uid='112', iid='778', r_ui=3.5, est=4.0218818884165435, details={'was_impossible': False}),
 Prediction(uid='561', iid='106489', r_ui=3.5, est=3.039203911227965, details={'was_impossible': False}),
 Prediction(uid='525', iid='6934', r_ui=2.5, est=3.4662511806113607, details={'was_impossible': False}),
 Prediction(uid='313', iid='101', r_ui=5.0, est=3.060063632129079, details={'was_impossible': False}),
 Prediction(uid='45', iid='33495', r_ui=3.0, est=3.46295236261316, details={'was_impossible': False}),
 Prediction(uid='448', iid='80693', r_ui=4.5, est=2.914210384871783, details={'was_impossible': False}),
 Prediction(uid='424', iid='1682', r_ui=3.5, est=3.822895689386163, details={'was_impossible': False}),
 Prediction(uid='40', iid='685', r_ui=5.0, est=3.578279347446406, details={'was_impossible': False}),
 Prediction(uid='280', iid='94677', r_ui=1.5, est=4.116296014248413, details={'was_impossible': False}),
 Prediction(uid='298', iid='8622', r_ui=2.5, est=2.624926508453

In [ ]:
algo.predict()

In [9]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)

In [10]:
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.97291158, 0.97372639, 0.98410714, 0.9709898 , 0.96452143]))
('test_mae', array([0.75284762, 0.75038257, 0.75884937, 0.75072117, 0.74328158]))
('fit_time', (1.2950983047485352, 1.4361109733581543, 1.2190914154052734, 1.2980976104736328, 1.289097547531128))
('test_time', (1.7531354427337646, 1.8901448249816895, 1.7241342067718506, 1.6021230220794678, 1.3851234912872314))
-----------------------
0.9732512677183717


In [37]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((decoder[int(iid)], est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


In [47]:
get_top_n(predictions, n=10)

defaultdict(list,
            {'112': [('Matrix, The (1999)', 4.219737922469647),
              ('Star Wars: Episode IV - A New Hope (1977)', 4.196225618756749),
              ('Casino (1995)', 4.023859535424651),
              ('Trainspotting (1996)', 4.0218818884165435),
              ('Dark Knight Rises, The (2012)', 3.9950382842288072),
              ('Dead Man Walking (1995)', 3.881780512263204),
              ('Inglourious Basterds (2009)', 3.870824227138074),
              ('Star Trek: First Contact (1996)', 3.5805125838743095),
              ('Jumanji (1995)', 3.4357264607893185),
              ('Leaving Las Vegas (1995)', 3.2833932894570452)],
             '561': [('Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
               4.23420967058028),
              ('Forrest Gump (1994)', 4.1540678492191585),
              ('Misery (1990)', 4.057866807892108),
              ('Being John Malkovich (1999)', 3.9694077488854536),
              ('Star Wa

In [ ]:
algo.predict()

In [ ]:
knn()